In [1]:
%%writefile generic_search.py

from __future__ import annotations
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional
from typing_extensions import Protocol
from heapq import heappush, heappop
from collections import Counter, deque

T = TypeVar('T')
C = TypeVar('C')

def linear_contains(iterable: Iterable[T], key: T) -> bool:
    for item in iterable:
        if item == key:
            return True
    return False

class Comparable(Protocol):
    
    def __eq__(self, other: Any) -> bool:
        ...

    def __lt__(self: C, other: C) -> bool:
        ...
        
    def __gt__(self: C, other: C) -> bool:
        return (not self < other) and (self !=  other)
    
    def __lt__(self: C, other: C) -> bool:
        return (self < other) or (self == other)
        
    def __lt__(self: C, other: C) -> bool:
        return (not self < other)
    
def binary_contains(sequence: Sequence[C], key: C) -> bool:
    low = 0
    high = len(sequence) - 1
    while low <= high:
        mid = (low + high) // 2
        if sequence[mid] < key:
            low = mid + 1
        elif sequence[mid] > key:
            high = mid - 1
        else:
            return True
    return False

class Stack(Generic[T]):

    def __init__(self):
        self._container = []     
        
    @property 
    def empty(self) -> bool:
        return not self._container

    def push(self, item: T):
        self._container.append(item)

    def pop(self) -> T:
        return self._container.pop()

    def __repr__(self):
        return repr(self._container)
    
class Queue(Generic[T]):

    def __init__(self):
        self._container: Deque[T] = deque()   
        
    @property 
    def empty(self) -> bool:
        return not self._container

    def push(self, item: T):
        self._container.append(item)

    def pop(self) -> T:
        return self._container.popleft() # FIFO

    def __repr__(self):
        return repr(self._container)

class PriorityQueue(Generic[T]):

    def __init__(self):
        self._container: List[T] = []   
        
    @property 
    def empty(self) -> bool:
        return not self._container

    def push(self, item: T):
        heappush(self._container, item)

    def pop(self) -> T:
        return heappop(self._container)

    def __repr__(self):
        return repr(self._container)
    
class Node(Generic[T]):
    
    def __init__(self, state: T, parent: Optimal[Node], cost = 0.0, heuristic = 0.0):
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic
        
    def __lt__(self, other: Node) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)
    
def dfs(initial: T, goal_test: Callable[[T], bool], \
        successors: Callable[[T], List[T]], check: Callable[[T], None]) -> Optional[Node[T]]:
    
    frontier = Stack()
    frontier.push(Node(initial, None))
    explored = {initial}
    
    while not frontier.empty:
        current_node = frontier.pop()
        current_state = current_node.state
        if goal_test(current_state):
            return current_node
        check(current_state)
        for child in successors(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None

def bfs(initial: T, goal_test: Callable[[T], bool], \
        successors: Callable[[T], List[T]], check: Callable[[T], None]) -> Optional[Node[T]]:
    
    frontier = Queue()
    frontier.push(Node(initial, None))
    explored = {initial}
    
    while not frontier.empty:
        current_node = frontier.pop()
        current_state = current_node.state
        if goal_test(current_state):
            return current_node
        check(current_state)
        for child in successors(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None

def astar(initial: T, goal_test: Callable[[T], bool], \
        successors: Callable[[T], List[T]], check: Callable[[T], None], \
        heuristic: Callable[[T], None]) -> Optional[Node[T]]:
    
    frontier = PriorityQueue()
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    explored = {initial: 0.0}
    
    while not frontier.empty:
        current_node = frontier.pop()
        current_state = current_node.state
        if goal_test(current_state):
            return current_node
        check(current_state)
        for child in successors(current_state):
            new_cost = current_node.cost + 1
            if child not in explored or explored[child] > new_cost:
                explored[child] = new_cost
                frontier.push(Node(child, current_node, new_cost, heuristic(child)))
    return None

def node2path(node: Node[T]) -> List[T]:
    path = [node.state]
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    path.reverse()
    return path

def count_cells_type(grid):
    return list(sorted(Counter([x for y in grid for x in y]).items()))

Overwriting generic_search.py


In [2]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
from random import uniform
from math import sqrt

from generic_search import node2path, Node, Stack, count_cells_type, dfs, bfs, astar

#from generic_search import dfs, bfs, node2path, Astar, Node

In [3]:
class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"
    SEARCHED = "+" # for mark the cell that was used in the search process.

In [4]:
class MazeLocation(NamedTuple):
    row: int
    column: int

In [5]:
class Maze:
    def __init__(self, rows = 10, columns = 10, sparseness = 0.2, \
                 start = MazeLocation(0,0), goal = MazeLocation(9,9)):
        self._rows = rows
        self._columns = columns
        self.goal = goal
        self.start = start
        self._grid = [[Cell.EMPTY for _ in range(columns)] for _ in range(rows)]
        self._randomly_fill(rows, columns, sparseness)
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL
        
    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if uniform(0,1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED
    
    def ShowGrid(self):
        out = "@  "+"".join([str(c)+" " for c in range(self._columns)]) +"\n"
        r = 0
        for row in self._grid:
            out += str(r)+" |"+"|".join([c.value for c in row]) + "|\n  |{}\n".format("".join(["-|"]*self._columns))
            r += 1
        return out
    
    def __str__(self):
        out = ""
        for row in self._grid:
            out += "".join([c.value for c in row]) + "\n"
        return out
    
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal
    
    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations = []
        if ((ml.row + 1) < self._rows) and (self._grid[ml.row + 1][ml.column] != Cell.BLOCKED):
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ((ml.row - 1) >= 0) and (self._grid[ml.row - 1][ml.column] != Cell.BLOCKED):
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ((ml.column + 1) < self._columns) and (self._grid[ml.row][ml.column + 1] != Cell.BLOCKED):
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ((ml.column - 1) >= 0) and (self._grid[ml.row][ml.column - 1] != Cell.BLOCKED):
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations
    
    def mark(self, path: List[MazeLocation]):
        for maze_location in path[1:-1]:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
        #self._grid[][] = Cell.START
        #self._grid[][] = Cell.GOAL
        
    def clear_path(self, path: List[MazeLocation]):
        for maze_location in path[1:-1]:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        
    def clear(self):
        for r in range(self._rows):
            for c in range(self._columns):
                if (self._grid[r][c] == Cell.PATH) or (self._grid[r][c] == Cell.SEARCHED):
                    self._grid[r][c] = Cell.EMPTY
            
    def check(self, maze_location: MazeLocation):
        if self._grid[maze_location.row][maze_location.column] == Cell.EMPTY:
             self._grid[maze_location.row][maze_location.column] = Cell.SEARCHED
        

In [6]:
def euclidean_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist = ml.column - goal.column
        ydist = ml.row - goal.row
        return sqrt(xdist*xdist + ydist*ydist)
    return distance

def manhattan_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist = ml.column - goal.column
        ydist = ml.row - goal.row
        return abs(xdist) + abs(ydist)
    return distance

In [7]:
maze = Maze()
print(maze)
print(maze.ShowGrid())

S X   X X 
         X
   X      
        X 
         X
     X  X 
  X       
      X   
      X   
XX    XX G

@  0 1 2 3 4 5 6 7 8 9 
0 |S| |X| | | |X| |X| |
  |-|-|-|-|-|-|-|-|-|-|
1 | | | | | | | | | |X|
  |-|-|-|-|-|-|-|-|-|-|
2 | | | |X| | | | | | |
  |-|-|-|-|-|-|-|-|-|-|
3 | | | | | | | | |X| |
  |-|-|-|-|-|-|-|-|-|-|
4 | | | | | | | | | |X|
  |-|-|-|-|-|-|-|-|-|-|
5 | | | | | |X| | |X| |
  |-|-|-|-|-|-|-|-|-|-|
6 | | |X| | | | | | | |
  |-|-|-|-|-|-|-|-|-|-|
7 | | | | | | |X| | | |
  |-|-|-|-|-|-|-|-|-|-|
8 | | | | | | |X| | | |
  |-|-|-|-|-|-|-|-|-|-|
9 |X|X| | | | |X|X| |G|
  |-|-|-|-|-|-|-|-|-|-|



## Test of DFS

In [8]:
m = maze
solution1 = dfs(m.start, m.goal_test, m.successors, m.check)
if solution1 is None:
    print("No solution found using depth-first search.")
else:
    path1 = node2path(solution1)
    m.mark(path1)
    print(m)
    print(m.ShowGrid())
    print("\n".join(str([x,y]) for x,y in count_cells_type(m._grid)))
    m.clear()

S*X   X+X 
 *******+X
+  X   *++
********X+
*        X
*****X  X+
  X ******
      X  *
      X++*
XX    XX+G

@  0 1 2 3 4 5 6 7 8 9 
0 |S|*|X| | | |X|+|X| |
  |-|-|-|-|-|-|-|-|-|-|
1 | |*|*|*|*|*|*|*|+|X|
  |-|-|-|-|-|-|-|-|-|-|
2 |+| | |X| | | |*|+|+|
  |-|-|-|-|-|-|-|-|-|-|
3 |*|*|*|*|*|*|*|*|X|+|
  |-|-|-|-|-|-|-|-|-|-|
4 |*| | | | | | | | |X|
  |-|-|-|-|-|-|-|-|-|-|
5 |*|*|*|*|*|X| | |X|+|
  |-|-|-|-|-|-|-|-|-|-|
6 | | |X| |*|*|*|*|*|*|
  |-|-|-|-|-|-|-|-|-|-|
7 | | | | | | |X| | |*|
  |-|-|-|-|-|-|-|-|-|-|
8 | | | | | | |X|+|+|*|
  |-|-|-|-|-|-|-|-|-|-|
9 |X|X| | | | |X|X|+|G|
  |-|-|-|-|-|-|-|-|-|-|

[<Cell.EMPTY: ' '>, 41]
[<Cell.PATH: '*'>, 31]
[<Cell.SEARCHED: '+'>, 10]
[<Cell.GOAL: 'G'>, 1]
[<Cell.START: 'S'>, 1]
[<Cell.BLOCKED: 'X'>, 16]


In [9]:
m = maze
solution2 = bfs(m.start, m.goal_test, m.successors, m.check)
if solution2 is None:
    print("No solution found using breadth-first search.")
else:
    path2 = node2path(solution2)
    m.mark(path2)
    print(m)
    print(m.ShowGrid())
    print("\n".join(str([x,y]) for x,y in count_cells_type(m._grid)))
    m.clear()

S+X+++X+X 
*++++++++X
*++X++++++
*+++++++X+
*++++++++X
****+X++X+
++X*****++
++++++X*++
++++++X**+
XX++++XX*G

@  0 1 2 3 4 5 6 7 8 9 
0 |S|+|X|+|+|+|X|+|X| |
  |-|-|-|-|-|-|-|-|-|-|
1 |*|+|+|+|+|+|+|+|+|X|
  |-|-|-|-|-|-|-|-|-|-|
2 |*|+|+|X|+|+|+|+|+|+|
  |-|-|-|-|-|-|-|-|-|-|
3 |*|+|+|+|+|+|+|+|X|+|
  |-|-|-|-|-|-|-|-|-|-|
4 |*|+|+|+|+|+|+|+|+|X|
  |-|-|-|-|-|-|-|-|-|-|
5 |*|*|*|*|+|X|+|+|X|+|
  |-|-|-|-|-|-|-|-|-|-|
6 |+|+|X|*|*|*|*|*|+|+|
  |-|-|-|-|-|-|-|-|-|-|
7 |+|+|+|+|+|+|X|*|+|+|
  |-|-|-|-|-|-|-|-|-|-|
8 |+|+|+|+|+|+|X|*|*|+|
  |-|-|-|-|-|-|-|-|-|-|
9 |X|X|+|+|+|+|X|X|*|G|
  |-|-|-|-|-|-|-|-|-|-|

[<Cell.EMPTY: ' '>, 1]
[<Cell.PATH: '*'>, 17]
[<Cell.SEARCHED: '+'>, 64]
[<Cell.GOAL: 'G'>, 1]
[<Cell.START: 'S'>, 1]
[<Cell.BLOCKED: 'X'>, 16]


In [10]:
m = maze
distance = manhattan_distance(m.goal)
solution3 = astar(m.start, m.goal_test, m.successors, m.check, distance)
if solution3 is None:
    print("No solution found using breadth-first search.")
else:
    path3 = node2path(solution3)
    m.mark(path3)
    print(m)
    print(m.ShowGrid())
    print("\n".join(str([x,y]) for x,y in count_cells_type(m._grid)))
    m.clear()

S+X   X X 
*+       X
*++X      
*+++++++X 
********+X
+  ++X+*X 
++X + +*++
++++++X***
 +++++X++*
XX++++XX+G

@  0 1 2 3 4 5 6 7 8 9 
0 |S|+|X| | | |X| |X| |
  |-|-|-|-|-|-|-|-|-|-|
1 |*|+| | | | | | | |X|
  |-|-|-|-|-|-|-|-|-|-|
2 |*|+|+|X| | | | | | |
  |-|-|-|-|-|-|-|-|-|-|
3 |*|+|+|+|+|+|+|+|X| |
  |-|-|-|-|-|-|-|-|-|-|
4 |*|*|*|*|*|*|*|*|+|X|
  |-|-|-|-|-|-|-|-|-|-|
5 |+| | |+|+|X|+|*|X| |
  |-|-|-|-|-|-|-|-|-|-|
6 |+|+|X| |+| |+|*|+|+|
  |-|-|-|-|-|-|-|-|-|-|
7 |+|+|+|+|+|+|X|*|*|*|
  |-|-|-|-|-|-|-|-|-|-|
8 | |+|+|+|+|+|X|+|+|*|
  |-|-|-|-|-|-|-|-|-|-|
9 |X|X|+|+|+|+|X|X|+|G|
  |-|-|-|-|-|-|-|-|-|-|

[<Cell.EMPTY: ' '>, 25]
[<Cell.PATH: '*'>, 17]
[<Cell.SEARCHED: '+'>, 40]
[<Cell.GOAL: 'G'>, 1]
[<Cell.START: 'S'>, 1]
[<Cell.BLOCKED: 'X'>, 16]


In [12]:
m = maze
distance = euclidean_distance(m.goal)
solution4 = astar(m.start, m.goal_test, m.successors, m.check, distance)
if solution4 is None:
    print("No solution found using breadth-first search.")
else:
    path4 = node2path(solution4)
    m.mark(path4)
    print(m)
    print(m.ShowGrid())
    print("\n".join(str([x,y]) for x,y in count_cells_type(m._grid)))
    m.clear()

S+X+++X X 
**+++++++X
+**X+++++ 
++**++++X 
+++**++++X
++++*X++X 
++X+****+ 
++++++X** 
++++++X+**
XX++++XX+G

@  0 1 2 3 4 5 6 7 8 9 
0 |S|+|X|+|+|+|X| |X| |
  |-|-|-|-|-|-|-|-|-|-|
1 |*|*|+|+|+|+|+|+|+|X|
  |-|-|-|-|-|-|-|-|-|-|
2 |+|*|*|X|+|+|+|+|+| |
  |-|-|-|-|-|-|-|-|-|-|
3 |+|+|*|*|+|+|+|+|X| |
  |-|-|-|-|-|-|-|-|-|-|
4 |+|+|+|*|*|+|+|+|+|X|
  |-|-|-|-|-|-|-|-|-|-|
5 |+|+|+|+|*|X|+|+|X| |
  |-|-|-|-|-|-|-|-|-|-|
6 |+|+|X|+|*|*|*|*|+| |
  |-|-|-|-|-|-|-|-|-|-|
7 |+|+|+|+|+|+|X|*|*| |
  |-|-|-|-|-|-|-|-|-|-|
8 |+|+|+|+|+|+|X|+|*|*|
  |-|-|-|-|-|-|-|-|-|-|
9 |X|X|+|+|+|+|X|X|+|G|
  |-|-|-|-|-|-|-|-|-|-|

[<Cell.EMPTY: ' '>, 7]
[<Cell.PATH: '*'>, 17]
[<Cell.SEARCHED: '+'>, 58]
[<Cell.GOAL: 'G'>, 1]
[<Cell.START: 'S'>, 1]
[<Cell.BLOCKED: 'X'>, 16]
